# Feb 5, 2025: high resolution parcellation, make rois contiguous

In [1]:
import csv 
import os
import json
import numpy as np
import pandas as pd
import dill as pickle
import nibabel as nib
from tqdm import tqdm
from glob import glob

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

np.random.seed(args.SEED)

In [3]:
BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
os.system(f'mkdir -p {PARCELS_path}')

0

In [ ]:
roi_names_file = f'{BASE_path}/parcellation/space-yale/mouseatlas_2020.json'
with open(roi_names_file, 'rb') as f:
    data = json.load(f)
for dt in data['labels']['data']:
    if dt['name'] == 'Mid-60':
        roi_names = dt['labels']

roi_names_df = pd.DataFrame(
    list(roi_names.items()), 
    columns=['old_roi', 'name'],
)
roi_names_df['old_roi'] = roi_names_df['old_roi'].astype(int)
roi_names_df

,old_roi,name
0,1,"R-Frontal pole, cerebral cortex (FRP,184)"
1,2,"R-Somatomotor areas (MO,500)"
2,3,"R-Somatosensory areas (SS,453)"
3,4,"R-Gustatory areas (GU,1057)"
4,5,"R-Visceral area (VISC,677)"
...,...,...
115,156,"L-Medulla, sensory related (MY-sen,386)"
116,157,"L-Medulla, motor related (MY-mot,370)"
117,158,"L-Medulla, behavioral state related (MY-sat,379)"
118,159,"L-Cerebellar cortex (CBX,528)"


In [5]:
RES = 200 #200 # 25
if RES == 25:
    old_parcels_file = (
        f'{BASE_path}/parcellation/space-allen'
        f'/medium_atlas2Allen_highres.nii'
    )
    old_template_file = (
        f'{BASE_path}/allen_atlas_ccfv2/atlasVolume'
        f'/average_template_25_LPS.nii.gz'
    )
if RES == 200:
    old_parcels_file = (
        f'{BASE_path}/parcellation/space-allen'
        f'/medium_atlas2Allen.nii'
    )
    old_template_file = (
        f'{BASE_path}/parcellation/space-allen'
        f'/Atlas2014_LPS_sm_resampled_0.2.nii'
    )

old_parcels_img = nib.load(old_parcels_file)
old_parcels = old_parcels_img.get_fdata()

In [6]:
res = np.abs(np.round(old_parcels_img.affine[0,0], decimals=3)) #mm
res = int(1000 * res) # um
res

200

In [7]:
unique_labels = np.unique(old_parcels)
unique_labels = unique_labels[unique_labels > 0] # remove background (zero)

new_labels = np.arange(1, len(unique_labels)+1)
label_mapping = dict(zip(unique_labels, new_labels))

parcels = np.zeros_like(old_parcels)
for old_label, label in tqdm(label_mapping.items()):
    parcels[old_parcels == old_label] = label

  0%|          | 0/120 [00:00<?, ?it/s]

100%|██████████| 120/120 [00:00<00:00, 3073.03it/s]


In [8]:
args.source = 'allen'
args.space = 'ccfv2'
args.brain_div = 'whl'
args.num_rois = len(label_mapping) # 120
args.resolution = res #200, 25 #um

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-120_res-200'

In [9]:
# parcellation volume
parcels_img = nib.Nifti1Image(
    parcels, 
    affine=old_parcels_img.affine, 
    header=old_parcels_img.header
)
nib.save(
    parcels_img, 
    f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz'
)

In [10]:
# roi labels
pd.options.mode.copy_on_write = True
roi_names_df = roi_names_df[roi_names_df['old_roi'].isin(list(label_mapping.keys()))]
roi_names_df['roi'] = roi_names_df['old_roi'].map(label_mapping)
roi_names_df.to_csv(
    f'{PARCELS_path}/{PARC_DESC}_desc-names.csv',
    index=False,
)

In [ ]:
# put the template in the PARCELS_path
TEMP_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_res-{args.resolution}'
)
template_file = f'{PARCELS_path}/{TEMP_DESC}_desc-template.nii.gz'

old_template_img = nib.load(old_template_file)
old_template = old_template_img.get_fdata()

nib.save(
    old_template_img,
    f'{template_file}',
)

# os.system(f'cp {old_template_file} {template_file}')

0